In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from scripts.indicateur import get_connexion_weight_in_synth_rotation as freq
from scripts.indicateur import get_typologie_rotation_CAN_synthetise as fnc
from scripts.indicateur import get_typologie_culture_CAN as typoC


In [2]:
# SCRIPT DE CREATION DE TU POUR LES DEUX FONCTION DE TYPOLOGIE CULTURE ET ROTATION
# A UTILISER A LA RACINE DE 02_OUTILS POUR FONCTIONNER !!!

In [3]:
# ------------------ #
# IMPORT DES DONNÉES #
# ------------------ #
ENTREPOT_PATH = '~/Bureau/Datagrosyst/data_entrepot_outils/'
EXTERN_DATA_PATH = '~/Bureau/Datagrosyst/data_entrepot_outils/referentiels/'
df = {}

def import_df(df_name, path_data, sep, index_col=None):
    df[df_name] = pd.read_csv(path_data+df_name+'.csv', sep = sep, index_col=index_col, low_memory=False).replace({'\r\n': '\n'}, regex=True)

def import_dfs(df_names, path_data, sep = ',', index_col=None, verbose=False):
    for df_name in tqdm(df_names) : 
        if(verbose) :
            print(' - ', df_name)
        import_df(df_name, path_data, sep, index_col=index_col)

tables_entrepot = [
    'connection_synthetise', 'noeuds_synthetise',
    'noeuds_synthetise_restructure','connection_synthetise_restructure'
]

data_pour_typologie_culutre = [
    'espece', 'culture', 'composant_culture','recolte_rendement_prix','recolte_rendement_prix_restructure'
]

extern_data_pour_typologie_culutre = [
    'typo_especes_typo_culture', 'typo_especes_typo_culture_marai'
]

# import des données de l'entrepôt avec la colonne 'id' en index 
import_dfs(tables_entrepot, ENTREPOT_PATH, sep = ',', verbose=False)

# import des données pour 'typologie_can_culture'
import_dfs(data_pour_typologie_culutre, ENTREPOT_PATH, sep = ',', verbose=False)
import_dfs(extern_data_pour_typologie_culutre, EXTERN_DATA_PATH, sep = ',', verbose=False)


nod = df['noeuds_synthetise']
nodres = df['noeuds_synthetise_restructure']
cnx = df['connection_synthetise']
cnxres = df['connection_synthetise_restructure']

rec = df['recolte_rendement_prix']
recrest = df['recolte_rendement_prix_restructure']

culture = df['culture']
cpc = df['composant_culture']
espece = df['espece']

100%|██████████| 2/2 [00:00<00:00, 472.65it/s]


In [4]:
# Filtrer sur quelques synthétisé
list_synth = [
    'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_c3c050a7-cedb-4071-9a0e-fadcbb311cc2',
    'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_b142d1f3-cb04-418c-bd77-31ac3922b09b',
    'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_85cedf6b-df0e-4470-b42b-1232608495aa',
    'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_933f1420-44ef-4809-aa4a-99c895b37554', #Betterave Fouragere
    'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_372ee474-d430-4e3c-8afc-f6814e7a1496' #Culture porte graine de Trefle
]

nod = nod.loc[nod['synthetise_id'].isin(list_synth)]
nodres = nodres.loc[nodres['id'].isin(nod['id'])]
cnx = cnx.loc[(cnx['cible_noeuds_synthetise_id'].isin(nod['id'])) | (cnx['source_noeuds_synthetise_id'].isin(nod['id']))]
cnxres = cnxres.loc[cnxres['id'].isin(cnx['id'])]

culture = culture.loc[(culture['id'].isin(nodres['culture_id'])) | (culture['id'].isin(cnxres['culture_intermediaire_id']))]
cpc = cpc.loc[cpc['culture_id'].isin(culture['id'])]
# Attention on ne filtre pas le référentiel espece, on veut le tester en entier

recrest = recrest.loc[recrest['composant_culture_id'].isin(cpc['id'])]
rec = rec.loc[rec['composant_culture_code'].isin(cpc['code'])]

In [5]:
df_filt = {
    'noeuds_synthetise': nod,
    'noeuds_synthetise_restructure': nodres,
    'connection_synthetise': cnx,
    'connection_synthetise_restructure':cnxres,
    'culture': culture,
    'composant_culture': cpc,
    'recolte_rendement_prix_restructure':recrest,
    'recolte_rendement_prix': rec,
    'espece': df['espece'],
    'typo_especes_typo_culture': df['typo_especes_typo_culture'],
    'typo_especes_typo_culture_marai': df['typo_especes_typo_culture_marai']

}

In [6]:
# Essaie des fonctions
# Fonction de typologie de culture
df_filt['typologie_can_culture'] = typoC(df_filt)
# Fonction de fréquence de connexions
df_filt['poids_connexions_synthetise_rotation'], _, _ = freq(df_filt, parallelization_enabled=False)
# Fonction de typologie de rotation
result = fnc(df_filt)

KeyError: 'composant_culture_id'

In [ ]:
PATH_TO_TEST_TYPOCULTURE = '~/Bureau/Datagrosyst/catalogue_script_agrosyst/02_outils/tests/data/test_get_typologie_culture_CAN/'

PATH_TO_TEST_TYPOROTATION = '~/Bureau/Datagrosyst/catalogue_script_agrosyst/02_outils/tests/data/test_get_typologie_rotation_CAN_synthetise/'

# for i in df_filt: 
#     if i in ['culture','composant_culture','espece','typo_especes_typo_culture','typo_especes_typo_culture_marai',
#              'recolte_rendement_prix_restructure','recolte_rendement_prix'] :
#         df_filt[i].to_csv(PATH_TO_TEST_TYPOCULTURE + str(i)+'.csv', index = False)

for i in df_filt: 
    if i in ['noeuds_synthetise_restructure','poids_connexions_synthetise_rotation','typologie_can_culture','noeuds_synthetise','connection_synthetise'] :
        df_filt[i].to_csv(PATH_TO_TEST_TYPOROTATION + str(i)+'.csv', index = False)

In [ ]:
df_filt['typologie_can_culture'].to_csv('~/Bureau/' + 'metadata_TU_typoC' + '.csv')
result.to_csv('~/Bureau/' + 'metadata_TU_typoRota' + '.csv')